## Hands-on - Virtualize NetCDF from ESGF

This example uses data from the [Earth System Grid Federation THREDDS Data Server](https://esgf-data.ucar.edu/thredds/catalog/catalog.html).

This is a quicker example for hands-on experience. For a full walkthrough, view previous example on the NASA-NEX-GDDP CMIP6 data.

Thank you to Raphael Hagen for contributing this example!

### Step 1: Import necessary functions and classes

In [ ]:
from obstore.store import HTTPStore
from virtualizarr import open_virtual_dataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry

### Step 2: Define data location

In [ ]:
bucket = "https://esgf-data.ucar.edu"
path = "thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/historical/r3i1p1f1/day/tas/gn/v20190308/tas_day_CESM2_historical_r3i1p1f1_gn_19200101-19291231.nc"

### Step 3: Create an ObjectStore and an ObjectStoreRegistry

In [ ]:
store = HTTPStore.from_url(bucket)
registry = ObjectStoreRegistry({bucket: store})

### Step 4: Create an instance of the HDFParser

In [ ]:
parser = HDFParser()

### Step 5: Create a virtual dataset via open_virtual_dataset

In [ ]:
vds = open_virtual_dataset(
    url=f"{bucket}/{path}",
    parser=parser,
    registry=registry,
    loadable_variables=["lat", "lon", "time", "time_bnds", "lat_bnds", "lon_bnds"],
)

In [ ]:
vds